<a href="https://colab.research.google.com/github/gala127/My_Important_Dates/blob/master/Copy_of_PyCode_5May.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#"""Консольный скрипт консультации по методу Суркова с хранением данных в SQLite.
#Простой запуск без дополнительных настроек:
#  1. Установите зависимость: pip install openai
#  2. Установите переменную окружения OPENAI_API_KEY или введите ключ при старте
#  3. Запустите: python consult_sqlite.py
#"""

import os
import sqlite3
import textwrap
import openai
from datetime import datetime
import re

# --- Настройки OpenAI ---
KEY = os.getenv("OPENAI_API_KEY")
if not KEY:
    KEY = input("Введите OPENAI_API_KEY: ").strip()
openai.api_key = KEY

# --- Путь к базе данных ---
DB_PATH = os.path.join(os.path.dirname(__file__), "consultation.sqlite3")

# --- Инициализация базы данных ---
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.executescript("""
    CREATE TABLE IF NOT EXISTS sessions (
        session_id TEXT PRIMARY KEY,
        ts_start   TEXT
    );
    CREATE TABLE IF NOT EXISTS messages (
        id         INTEGER PRIMARY KEY AUTOINCREMENT,
        session_id TEXT,
        step       INTEGER,
        question   TEXT,
        answer     TEXT,
        ts         TEXT
    );
    CREATE TABLE IF NOT EXISTS problems (
        session_id TEXT PRIMARY KEY,
        nickname   TEXT,
        formula    TEXT,
        stability  TEXT,
        plan       TEXT
    );
    """)
    conn.commit()
    conn.close()

# --- Логирование ---
def log_session(session_id):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    now = datetime.utcnow().isoformat()
    cur.execute("""
        INSERT OR IGNORE INTO sessions(session_id, ts_start)
        VALUES(?,?)
    """, (session_id, now))
    conn.commit()
    conn.close()

def log_message(session_id, step, question, answer):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    now = datetime.utcnow().isoformat()
    cur.execute("""
        INSERT INTO messages(session_id, step, question, answer, ts)
        VALUES(?,?,?,?,?)
    """, (session_id, step, question, answer, now))
    conn.commit()
    conn.close()

def log_problem_field(session_id, field, value):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute(f"""
        INSERT OR IGNORE INTO problems(session_id) VALUES(?);
    """, (session_id,))
    cur.execute(f"UPDATE problems SET {field} = ? WHERE session_id = ?;", (value, session_id))
    conn.commit()
    conn.close()

# --- Интеграция с OpenAI для уточнений ---
def generate_followup(prev_q, answer, step_no):
    prompt = (
        f"Шаг {step_no}. Клиент ответил: \"{answer}\" на вопрос: \"{prev_q}\".\n"
        "Сгенерируй один уточняющий вопрос, чтобы сделать ответ более конкретным."
    )
    resp = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Генерируй уточняющие вопросы."},
            {"role": "user",   "content": prompt}
        ]
    )
    return resp.choices[0].message.content.strip()

# --- Вспомогательные функции ---
def prompt_user(msg):
    ans = input(f"{msg}\n> ").strip()
    return ans

def clarify(session_id, step_no, question):
    while True:
        ans = prompt_user(question)
        log_message(session_id, step_no, question, ans)
        if len(ans.split()) >= 3:
            return ans
        follow = generate_followup(question, ans, step_no)
        print("Уточнение:", follow)
        question = follow

# --- Шаги методики ---
def step1(session_id):
    print("\n=== ШАГ 1: Формулировка проблемы ===")
    nickname = prompt_user("Здравствуйте! Представьтесь, пожалуйста.")
    log_session(session_id)
    log_problem_field(session_id, "nickname", nickname)
    print("\n=== Анализ ===")
    questions = [
        ("Что происходит не так, как вам хотелось бы?", "what"),
        ("Где именно это происходит? Уточните место.",   "where"),
        ("Когда именно это происходит? Уточните время/условия.", "when"),
        ("Почему, по вашему, это происходит? (первые причины)",  "why_first")
    ]
    ctx = {}
    for i, (q, key) in enumerate(questions, start=5):
        ans = clarify(session_id, i, q)
        ctx[key] = ans
    formula = f"{ctx['when']} в {ctx['where']} {ctx['what']}, потому что {ctx['why_first']}"
    print("\n— Формула проблемы:")
    print(textwrap.fill(formula, width=70))
    log_problem_field(session_id, "formula", formula)
    return ctx
    return nickname

def step2(session_id):
    print("\n=== ШАГ 2: Поиск простых решений ===")
    # Отказ от ситуации
    q1 = "Можно ли избежать ситуации — отказаться от действия, приводящего к проблеме?"
    a1 = prompt_user(q1); log_message(session_id, 2, q1, a1)
    if a1.lower().startswith("да"):
        log_problem_field(session_id, "stability", "решено отказом")
        return True
    # Принятие/минимизация
    q2 = "Можно ли смириться с ситуацией или минимизировать её влияние?"
    a2 = prompt_user(q2); log_message(session_id, 3, q2, a2)
    # Известные решения
    q3 = "Какие стандартные решения вы пробовали или знаете?"
    a3 = prompt_user(q3); log_message(session_id, 4, q3, a3)
    solved = any(ans.lower().startswith("да") for ans in (a1, a2, a3))
    if solved:
        log_problem_field(session_id, "stability", "решено простыми методами")
        return True
    print("Простые решения не устранили проблему — переходим дальше.")
    return False
#

def step3(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("SELECT formula FROM problems WHERE session_id = ?", (session_id,))
    row = cur.fetchone()
    if not row:
        print("Ошибка: сессия не найдена.")
        conn.close()
        return
    formula = row[0]
    print("\n=== Шаг 3: Анализ корневых причин («5 почему») ===")
    parts = re.split(r'[;,]| и | & ', formula)[:3]
    all_causes = {}
    # ... rest of step3 function ...

def step4(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    print("\n=== Шаг 4: Анализ ресурсов и условий ситуации ===")
    # ... rest of step4 function ...

def step5(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # ... rest of step5 function ...

def step6(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # ... rest of step6 function ...

def step7(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # ... rest of step7 function ...

def step8(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)



# --- Шаги методики ---
def step1(session_id):
    print("\n=== ШАГ 1: Формулировка проблемы ===")
    nickname = prompt_user("Здравствуйте! Представьтесь, пожалуйста.")
    log_session(session_id)
    log_problem_field(session_id, "nickname", nickname)
    print("\n=== Анализ ===")
    questions = [
        ("Что происходит не так, как вам хотелось бы?", "what"),
        ("Где именно это происходит? Уточните место.",   "where"),
        ("Когда именно это происходит? Уточните время/условия.", "when"),
        ("Почему, по вашему, это происходит? (первые причины)",  "why_first")
    ]
    ctx = {}
    for i, (q, key) in enumerate(questions, start=5):
        ans = clarify(session_id, i, q)
        ctx[key] = ans
    formula = f"{ctx['when']} в {ctx['where']} {ctx['what']}, потому что {ctx['why_first']}"
    print("\n— Формула проблемы:")
    print(textwrap.fill(formula, width=70))
    log_problem_field(session_id, "formula", formula)
    return ctx

def step2(session_id):
    print("\n=== ШАГ 2: Поиск простых решений ===")
    # Отказ от ситуации
    q1 = "Можно ли избежать ситуации — отказаться от действия, приводящего к проблеме?"
    a1 = prompt_user(q1); log_message(session_id, 2, q1, a1)
    if a1.lower().startswith("да"):
        log_problem_field(session_id, "stability", "решено отказом")
        return True
    # Принятие/минимизация
    q2 = "Можно ли смириться с ситуацией или минимизировать её влияние?"
    a2 = prompt_user(q2); log_message(session_id, 3, q2, a2)
    # Известные решения
    q3 = "Какие стандартные решения вы пробовали или знаете?"
    a3 = prompt_user(q3); log_message(session_id, 4, q3, a3)
    solved = any(ans.lower().startswith("да") for ans in (a1, a2, a3))
    if solved:
        log_problem_field(session_id, "stability", "решено простыми методами")
        return True
    print("Простые решения не устранили проблему — переходим дальше.")
    return False

def step3(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("SELECT formula FROM problems WHERE session_id = ?", (session_id,))
    row = cur.fetchone()
    if not row:
        print("Ошибка: сессия не найдена.")
        conn.close()
        return
    formula = row[0]
    print("\n=== Шаг 3: Анализ корневых причин («5 почему») ===")
    parts = re.split(r'[;,]| и | & ', formula)[:3]
    all_causes = {}
    for idx, reason in enumerate(parts, start=1):
        print(f"\n--- Ветвь {idx}: {reason} ---")
        causes = process_branch(conn, session_id, reason)
        all_causes[f"branch_{idx}"] = causes

    print("\n Итоговые корневые причины по ветвям:")
    for branch, causes in all_causes.items():
        print(f"{branch}: {', '.join(causes)}")

    conn.close()
    return all_causes


def step4(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    print("\n=== Шаг 4: Анализ ресурсов и условий ситуации ===")
    # ... rest of step4 function ...

def step5(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # ... rest of step5 function ...

def step6(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # ... rest of step6 function ...

def step7(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # ... rest of step7 function ...

def step8(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)

import sqlite3
import re
from datetime import datetime

def log_message(conn, session_id, question, answer, step=3):
    """
    Записывает вопрос и ответ в таблицу messages.
    """
    ts = datetime.utcnow().isoformat()
    conn.execute(
        "INSERT INTO messages(session_id, step, question, answer, ts) "
        "VALUES (?, ?, ?, ?, ?)",
        (session_id, step, question, answer, ts)
    )
    conn.commit()

def ask(question):
    """
    Задаёт вопрос клиенту и возвращает непустой ответ.
    """
    while True:
        ans = input(f"{question}\n> ").strip()
        if ans:
            return ans
        print("Пожалуйста, введите ответ.")

def is_base_cause():
    """
    Спрашивает, является ли текущий ответ базовой причиной.
    """
    resp = input("Это базовая причина? (да/нет)\n> ").strip().lower()
    return resp.startswith(("д", "y"))

def split_reasons(answer):
    """
    Разбивает ответ на несколько причин по разделителям.
    """
    parts = re.split(r'[;,]| и | & ', answer)
    return [p.strip() for p in parts if p.strip()]

def process_branch(conn, session_id, initial_reason,
                   max_whys=7, max_branch_whys=5):
    """
    Обрабатывает одну ветвь «почему» до max_whys итераций.
    Если ответ содержит несколько причин, для каждой заводит вложенную ветвь
    до max_branch_whys итераций.
    Возвращает список выявленных корневых причин для этой ветви.
    """
    final_causes = []
    stack = [(initial_reason, 1)]

    while stack:
        reason, depth = stack.pop()
        if depth > max_whys:
            final_causes.append(reason)
            continue

        q = f"Почему {reason}?"
        a = ask(q)
        log_message(conn, session_id, q, a)

        sub_reasons = split_reasons(a)
        if len(sub_reasons) > 1:
            # для каждой под-причины отдельная подветвь
            for sub in sub_reasons:
                sub_stack = [(sub, 1)]
                while sub_stack:
                    r, d = sub_stack.pop()
                    if d > max_branch_whys:
                        final_causes.append(r)
                        continue
                    sq = f"Почему {r}?"
                    sa = ask(sq)
                    log_message(conn, session_id, sq, sa)
                    if is_base_cause():
                        final_causes.append(sa)
                        break
                    for nr in split_reasons(sa):
                        sub_stack.append((nr, d+1))
        else:
            # одиночная ветвь
            if is_base_cause():
                final_causes.append(a)
                continue
            # углубляемся дальше
            stack.append((a, depth+1))

    return final_causes

def step3(session_id, db_path="consultation.db"):
    """
    Шаг 3 методики Суркова: анализ корневых причин («5 почему» с ветвлением).
    1) Берёт из problems.formula исходную формулировку.
    2) Разбивает её на до 3 частей (ветвей).
    3) Для каждой ветви вызывает process_branch.
    4) Выводит итоговые корневые причины всех ветвей.
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # Получаем формулу проблемы
    cur.execute("SELECT formula FROM problems WHERE session_id = ?", (session_id,))
    row = cur.fetchone()
    if not row:
        print("Ошибка: сессия не найдена.")
        conn.close()
        return

    formula = row[0]
    print("\n=== Шаг 3: Анализ корневых причин («5 почему») ===")
    # разбиваем исходную формулу на до 3 ветвей
    parts = split_reasons(formula)[:3]

    all_causes = {}
    for idx, reason in enumerate(parts, start=1):
        print(f"\n--- Ветвь {idx}: {reason} ---")
        causes = process_branch(conn, session_id, reason)
        all_causes[f"branch_{idx}"] = causes

    print("\n Итоговые корневые причины по ветвям:")
    for branch, causes in all_causes.items():
        print(f"{branch}: {', '.join(causes)}")

    conn.close()
    return all_causes

if __name__ == "__main__":
    sid = input("Введите идентификатор сессии:\n> ").strip()
    step3(sid)


def step4(session_id):
    print("\n=== ШАГ4: Ресурсы ===")

    # Шаг 4: Ресурсы
    q = "Перечислите через запятую объекты, людей и условия в том месте, где происходят нежелательные явления"
import sqlite3
from datetime import datetime

def log_message(conn, session_id, question, answer, step=4):
    """
    Записывает вопрос и ответ в таблицу messages.
    """
    ts = datetime.utcnow().isoformat()
    conn.execute(
        "INSERT INTO messages(session_id, step, question, answer, ts) "
        "VALUES (?, ?, ?, ?, ?)",
        (session_id, step, question, answer, ts)
    )
    conn.commit()

def ask_list(prompt):
    """
    Задаёт вопрос и возвращает список элементов, разделённых запятыми.
    """
    answer = input(f"{prompt}\n> ").strip()
    return [item.strip() for item in answer.split(",") if item.strip()]

def step4(session_id, db_path="consultation.db"):
    """
    Шаг 4: Анализ ресурсов и условий ситуации.
    Спрашивает по категориям и сохраняет каждый ответ в БД.
    """
    conn = sqlite3.connect(db_path)
    print("\n=== Шаг 4: Анализ ресурсов и условий ситуации ===")

    categories = [
        ("Основные действующие объекты",
         "Опишите основные действующие объекты в том месте, где происходят нежелательные явления "
         "(например, ваш автомобиль; автомобили соседей; сами парковочные места)"),
        ("Близлежащие объекты",
         "Опишите близлежащие объекты (например, тротуар вдоль дома; газон рядом; детская площадка; "
         "мусорные баки; подъезд; ворота/шлагбаум)"),
        ("Люди",
         "Опишите людей в оперативной зоне (например, Вы; соседи-автовладельцы; дворник; охрана)"),
        ("Правила/ограничения",
         "Какие есть правила и ограничения в том месте, где происходят нежелательные явления "
         "(например, \"Запрет парковаться на газоне/площадке\"; ПДД; часы, когда двор открыт)"),
        ("Природные и технические факторы",
         "Опишите, какие природные и технические факторы имеют значение там и тогда, "
         "когда происходят нежелательные явления (например, \"Время года; освещение; погода; снег; гравитация\")")
    ]


    for category, prompt in categories:
        # Отображаем полный prompt, включая примеры
        items = ask_list(prompt)
        # Логируем только часть prompt до "(например"
        log_question = prompt.split("(например")[0].strip()
        for item in items:
            log_message(conn, session_id, log_question, item, step=4)

    conn.close()
    print("Все ресурсы записаны в базу данных.")

    # Шаг 5: Идеальные решения

    ideals = {}
    for r in resources:
        iq = f"Пусть {r} самостоятельно решит проблему — как это было бы? Например, сосулька не упадет на голову, а растает или застрянет в водосточной трубе."
        ans = prompt_user(iq); log_message(session_id, 10, iq, ans)
        ideals[r] = ans

    import sqlite3
from datetime import datetime

def log_message(conn, session_id, question, answer, step=5):
    """
    Записывает вопрос и ответ в таблицу messages для шага 5.
    """
    ts = datetime.utcnow().isoformat()
    conn.execute(
        "INSERT INTO messages(session_id, step, question, answer, ts) "
        "VALUES (?, ?, ?, ?, ?)",
        (session_id, step, question, answer, ts)
    )
    conn.commit()

def ask_ideal(resource):
    """
    Спрашивает у клиента идеальное решение для данного ресурса.
    """
    prompt = (
        f"Представьте, что ресурс «{resource}» сам устраняет проблему. "
        "Сформулируйте идеальную фразу, начиная с «Пусть ...»."
    )
    while True:
        ans = input(f"{prompt}\n> ").strip()
        if ans.startswith("Пусть"):
            return ans
        print("Фраза должна начинаться с «Пусть». Попробуйте ещё раз.")

def step5(session_id, db_path="consultation.db"):
    """
    Шаг 5: Формулирование идеального решения.
    Для каждого ресурса, записанного на шаге 4, запрашивает у клиента идеальную формулировку:
    "Пусть {resource} сам ...", и сохраняет ответы в БД (step=5).
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # Получаем список ресурсов из Шага 4 (step=4), исключая оперативную зону
    cur.execute(
        "SELECT answer FROM messages "
        "WHERE session_id = ? AND step = 4 AND question != 'Оперативная зона'",
        (session_id,)
    )
    resources = [row[0] for row in cur.fetchall()]

    print("\n=== Шаг 5: Формулирование идеального решения ===")
    for resource in resources:
        ideal = ask_ideal(resource)
        log_message(conn, session_id, f"Идеальное решение для {resource}", ideal)

    conn.close()
    print("Все идеальные решения сохранены в базе данных.")


# Пример использования:
if __name__ == "__main__":
    sid = input("Введите идентификатор сессии:\n> ").strip()
    step5(sid)



def log_message(conn, session_id, question, answer, step=6):
    """
    Записывает вопрос и ответ в таблицу messages для шага 6.
    """
    ts = datetime.utcnow().isoformat()
    conn.execute(
        "INSERT INTO messages(session_id, step, question, answer, ts) "
        "VALUES (?, ?, ?, ?, ?)",
        (session_id, step, question, answer, ts)
    )
    conn.commit()

def ask(question):
    """
    Задаёт вопрос клиенту и возвращает непустой ответ.
    """
    while True:
        ans = input(f"{question}\n> ").strip()
        if ans:
            return ans
        print("Пожалуйста, введите ответ.")

def step6(session_id, db_path="consultation.db"):
    """
    Шаг 6: Выявление и формулирование противоречий.
    Для каждой идеальной фразы из Шага 5:
      1) спрашиваем, что мешает её реализации,
      2) просим сформулировать противоречие в формате:
         "Нужно X, но нельзя X, потому что B",
      и логирует оба ответа в БД.
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # Получаем все идеальные решения из Шага 5
    cur.execute(
        "SELECT id, answer FROM messages "
        "WHERE session_id = ? AND step = 5",
        (session_id,)
    )
    ideals = cur.fetchall()  # list of tuples (id, ideal_text)

    print("\n=== Шаг 6: Формулирование противоречий ===")
    for msg_id, ideal in ideals:
        print(f"\nИдея: {ideal}")

        # 1) Узнаём препятствие
        obstacle = ask(f"Что мешает реализовать эту идею?")
        log_message(conn, session_id, f"Препятствие для '{ideal}'", obstacle, step=6)

        # 2) Формулируем противоречие
        contradiction = ask(
            "Сформулируйте, пожалуйста, противоречие в формате:\n"
            "\"Нужно X, но нельзя X, потому что B\"\n"
            "(замените X и B своими словами)."
        )
        log_message(conn, session_id, f"Противоречие для '{ideal}'", contradiction, step=6)

    conn.close()
    print("\nВсе противоречия сохранены в базе данных. Переход к Шагу 7.")


# Пример использования:
if __name__ == "__main__":
    sid = input("Введите идентификатор сессии:\n> ").strip()
    step6(sid)

 # Код step6 написан так, чтобы работать в обычном терминале или в среде, где можно вводить ответы с клавиатуры.
# Что делать дальше:

# Скопируйте этот скрипт на свой компьютер и запустите его в консоли (например, python my_consultation.py),
# где input() будет работать как надо.

# Шаг 7: Поиск решений через разрешение противоречий

import sqlite3
from datetime import datetime

def log_message(conn, session_id, question, answer, step=7):
    """
    Записывает вопрос и ответ в таблицу messages для шага 7.
    """
    ts = datetime.utcnow().isoformat()
    conn.execute(
        "INSERT INTO messages(session_id, step, question, answer, ts) "
        "VALUES (?, ?, ?, ?, ?)",
        (session_id, step, question, answer, ts)
    )
    conn.commit()

def ask(question):
    """
    Задаёт вопрос и возвращает непустой ответ.
    """
    while True:
        ans = input(f"{question}\n> ").strip()
        if ans:
            return ans
        print("Пожалуйста, введите ответ.")

def step7(session_id, db_path="consultation.db"):
    """
    Шаг 7: Поиск решений через разрешение противоречий.
    1) Извлекает все сформулированные на шаге 6 противоречия.
    2) Для каждого противоречия задаёт вопрос:
       «Как можно добиться X, не нарушая условие B?»
       где X и B извлекаются из текста противоречия.
    3) Логирует каждый Q&A в БД (step=7).
    4) Выводит список всех предложенных решений.
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # Выбираем из messages все записи-шаги 6, где question начинается с "Противоречие"
    cur.execute("""
        SELECT id, answer
          FROM messages
         WHERE session_id = ?
           AND step = 6
           AND question LIKE 'Противоречие%'
    """, (session_id,))
    contradictions = cur.fetchall()

    if not contradictions:
        print("Не найдено ни одного противоречия для обработки.")
        conn.close()
        return []

    solutions = []
    print("\n=== Шаг 7: Поиск решений — «Как добиться X, не нарушая B?» ===")
    for msg_id, contradiction in contradictions:
        # Предполагаем, что текст противоречия вида:
        # "Нужно X, но нельзя X, потому что B"
        # Разбиваем на X и B
        parts = contradiction.split(" потому что ")
        if len(parts) == 2:
            left, barrier = parts
            # извлечь X из первой части после "Нужно "
            x_part = left.replace("Нужно ", "").split(", но")[0].strip()
        else:
            # fallback: весь текст используем как X, без B
            x_part = contradiction
            barrier = ""

        prompt = (
            f"Для противоречия:\n"
            f"  «{contradiction}»\n"
            f"Как можно добиться «{x_part}», "
            f"{'не нарушая условия: ' + barrier if barrier else ''}?"
        )
        sol = ask(prompt)
        log_message(conn, session_id, prompt, sol)
        solutions.append(sol)

    conn.close()

    print("\nСгенерированные идеи решений:")
    for i, s in enumerate(solutions, 1):
        print(f"{i}) {s}")

    return solutions

# Пример использования:
if __name__ == "__main__":
    sid = input("Введите идентификатор сессии:\n> ").strip()
    step7(sid)


# Шаг 8: противоречия и реальные решения

    solutions = {}
    step_no = 11
    for r, idea in ideals.items():
        cq = f"Почему нельзя: «{idea}»?"
        ca = prompt_user(cq); log_message(session_id, step_no, cq, ca)
        step_no += 1
        sq = f"Как добиться «{idea}», несмотря на: «{ca}»?"
        sa = prompt_user(sq); log_message(session_id, step_no, sq, sa)
        solutions[r] = sa
        step_no += 1
    # Выбор и план
    print("\nВарианты решений:")
    for idx, sol in enumerate(solutions.values(), 1):
        print(f"{idx}) {sol}")
    choice = int(prompt_user("Введите номер наиболее перспективного варианта")) - 1
    selected = list(solutions.values())[choice]
    plan = prompt_user(f"Опишите план действий для: «{selected}»")
    log_problem_field(session_id, "plan", plan)
    log_problem_field(session_id, "stability", "решено творчески")
    print("\nПлан внедрения сохранён.")
    return selected, plan

# Шаг 8: Выбор решения и план внедрения
import sqlite3
from datetime import datetime

def log_message(conn, session_id, question, answer, step=8):
    """
    Записывает вопрос/комментарий и ответ в таблицу messages для Шага 8.
    """
    ts = datetime.utcnow().isoformat()
    conn.execute(
        "INSERT INTO messages(session_id, step, question, answer, ts) "
        "VALUES (?, ?, ?, ?, ?)",
        (session_id, step, question, answer, ts)
    )
    conn.commit()

def ask(prompt):
    """
    Задаёт вопрос и возвращает непустой ответ.
    """
    while True:
        ans = input(f"{prompt}\n> ").strip()
        if ans:
            return ans
        print("Пожалуйста, введите ответ.")

def step8(session_id, db_path="consultation.db"):
    """
    Шаг 8: Выбор решения и план внедрения.
    1) Извлекает все идеи-решения (step=7) из БД.
    2) Предлагает клиенту выбрать одно (или несколько) лучших.
    3) Логирует выбранное решение.
    4) Просит описать пошаговый план реализации:
       - Задаёт вопросы «Опишите шаг 1», «Опишите шаг 2»… до пустого ввода.
       - Логирует каждый шаг как отдельное сообщение.
    5) Спрашивает про возможные трудности/подзадачи и логирует их.
    """
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # 1. Извлечь все решения из шага 7
    cur.execute(
        "SELECT id, answer FROM messages "
        "WHERE session_id = ? AND step = 7",
        (session_id,)
    )
    rows = cur.fetchall()
    if not rows:
        print("Не найдено ни одного решения для выбора (step=7).")
        conn.close()
        return

    # 2. Показать варианты и выбрать
    print("\n=== Шаг 8: Выбор решения и план внедрения ===")
    print("Доступные идеи-решения:")
    for idx, (msg_id, sol) in enumerate(rows, start=1):
        print(f"{idx}. {sol}")
    choice = None
    while choice not in range(1, len(rows)+1):
        try:
            choice = int(input(f"Выберите номер решения (1–{len(rows)}):\n> ").strip())
        except ValueError:
            continue
    chosen_id, chosen_text = rows[choice-1]
    log_message(conn, session_id, "Выбранное решение", chosen_text, step=8)
    print(f"Вы выбрали: {chosen_text}")

    # 3. Составить план реализации
    print("\nВведите по одному шагу плана реализации. Для завершения оставьте строку пустой.")
    step_num = 1
    while True:
        step_desc = input(f"Шаг {step_num}:\n> ").strip()
        if not step_desc:
            break
        log_message(conn, session_id, f"План: шаг {step_num}", step_desc, step=8)
        step_num += 1

    # 4. Вопрос о потенциальных трудностях
    obstacles = ask(
        "Какие потенциальные трудности или подзадачи могут возникнуть при реализации этого плана?"
    )
    log_message(conn, session_id, "Возможные трудности при внедрении", obstacles, step=8)

    conn.close()
    print("\nПлан внедрения сохранён в базе данных.")


# --- Главная функция ---
def main():
    init_db()

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#"""Консольный скрипт консультации по методу Суркова с хранением данных в SQLite.
#Простой запуск без дополнительных настроек:
#  1. Установите зависимость: pip install openai
#  2. Установите переменную окружения OPENAI_API_KEY или введите ключ при старте
#  3. Запустите: python consult_sqlite.py
#"""

import os
import sqlite3
import textwrap
import openai
from datetime import datetime
import re

# --- Настройки OpenAI ---
KEY = os.getenv("OPENAI_API_KEY")
if not KEY:
    KEY = input("Введите OPENAI_API_KEY: ").strip()
openai.api_key = KEY

# --- Путь к базе данных ---
DB_PATH = os.path.join(os.path.dirname(__file__), "consultation.sqlite3")

# --- Инициализация базы данных ---
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.executescript("""
    CREATE TABLE IF NOT EXISTS sessions (
        session_id TEXT PRIMARY KEY,
        ts_start   TEXT
    );
    CREATE TABLE IF NOT EXISTS messages (
        id         INTEGER PRIMARY KEY AUTOINCREMENT,
        session_id TEXT,
        step       INTEGER,
        question   TEXT,
        answer     TEXT,
        ts         TEXT
    );
    CREATE TABLE IF NOT EXISTS problems (
        session_id TEXT PRIMARY KEY,
        nickname   TEXT,
        formula    TEXT,
        stability  TEXT,
        plan       TEXT
    );
    """)
    conn.commit()
    conn.close()

# --- Логирование ---
def log_session(session_id):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    now = datetime.utcnow().isoformat()
    cur.execute("""
        INSERT OR IGNORE INTO sessions(session_id, ts_start)
        VALUES(?,?)
    """, (session_id, now))
    conn.commit()
    conn.close()

def log_message(session_id, step, question, answer):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    now = datetime.utcnow().isoformat()
    cur.execute("""
        INSERT INTO messages(session_id, step, question, answer, ts)
        VALUES(?,?,?,?,?)
    """, (session_id, step, question, answer, now))
    conn.commit()
    conn.close()

def log_problem_field(session_id, field, value):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute(f"""
        INSERT OR IGNORE INTO problems(session_id) VALUES(?);
    """, (session_id,))
    cur.execute(f"UPDATE problems SET {field} = ? WHERE session_id = ?;", (value, session_id))
    conn.commit()
    conn.close()

# --- Интеграция с OpenAI для уточнений ---
def generate_followup(prev_q, answer, step_no):
    prompt = (
        f"Шаг {step_no}. Клиент ответил: \"{answer}\" на вопрос: \"{prev_q}\".\n"
        "Сгенерируй один уточняющий вопрос, чтобы сделать ответ более конкретным."
    )
    resp = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Генерируй уточняющие вопросы."},
            {"role": "user",   "content": prompt}
        ]
    )
    return resp.choices[0].message.content.strip()

# --- Вспомогательные функции ---
def prompt_user(msg):
    ans = input(f"{msg}\n> ").strip()
    return ans

def clarify(session_id, step_no, question):
    while True:
        ans = prompt_user(question)
        log_message(session_id, step_no, question, ans)
        if len(ans.split()) >= 3:
            return ans
        follow = generate_followup(question, ans, step_no)
        print("Уточнение:", follow)
        question = follow

# --- Шаги методики ---
def step1(session_id):
    print("\n=== ШАГ 1: Формулировка проблемы ===")
    nickname = prompt_user("Здравствуйте! Представьтесь, пожалуйста.")
    log_session(session_id)
    log_problem_field(session_id, "nickname", nickname)
    print("\n=== Анализ ===")
    questions = [
        ("Что происходит не так, как вам хотелось бы?", "what"),
        ("Где именно это происходит? Уточните место.",   "where"),
        ("Когда именно это происходит? Уточните время/условия.", "when"),
        ("Почему, по вашему, это происходит? (первые причины)",  "why_first")
    ]
    ctx = {}
    for i, (q, key) in enumerate(questions, start=5):
        ans = clarify(session_id, i, q)
        ctx[key] = ans
    formula = f"{ctx['when']} в {ctx['where']} {ctx['what']}, потому что {ctx['why_first']}"
    print("\n— Формула проблемы:")
    print(textwrap.fill(formula, width=70))
    log_problem_field(session_id, "formula", formula)
    return ctx

def step2(session_id):
    print("\n=== ШАГ 2: Поиск простых решений ===")
    # Отказ от ситуации
    q1 = "Можно ли избежать ситуации — отказаться от действия, приводящего к проблеме?"
    a1 = prompt_user(q1); log_message(session_id, 2, q1, a1)
    if a1.lower().startswith("да"):
        log_problem_field(session_id, "stability", "решено отказом")
        return True
    # Принятие/минимизация
    q2 = "Можно ли смириться с ситуацией или минимизировать её влияние?"
    a2 = prompt_user(q2); log_message(session_id, 3, q2, a2)
    # Известные решения
    q3 = "Какие стандартные решения вы пробовали или знаете?"
    a3 = prompt_user(q3); log_message(session_id, 4, q3, a3)
    solved = any(ans.lower().startswith("да") for ans in (a1, a2, a3))
    if solved:
        log_problem_field(session_id, "stability", "решено простыми методами")
        return True
    print("Простые решения не устранили проблему — переходим дальше.")
    return False

def step3(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute("SELECT formula FROM problems WHERE session_id = ?", (session_id,))
    row = cur.fetchone()
    if not row:
        print("Ошибка: сессия не найдена.")
        conn.close()
        return
    formula = row[0]
    print("\n=== Шаг 3: Анализ корневых причин («5 почему») ===")
    parts = re.split(r'[;,]| и | & ', formula)[:3]
    all_causes = {}
    # ... rest of step3 function ...

def step4(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    print("\n=== Шаг 4: Анализ ресурсов и условий ситуации ===")
    # ... rest of step4 function ...

def step5(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # ... rest of step5 function ...

def step6(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # ... rest of step6 function ...

def step7(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # ... rest of step7 function ...

def step8(session_id, db_path="consultation.db"):
    conn = sqlite3.connect(db_path)

def main():
    init_db()
    session_id = prompt_user("Введите идентификатор сессии")
    step1(session_id)
    if not step2(session_id):
        step3(session_id)
        step4(session_id)
        step5(session_id)
        step6(session_id)
        step7(session_id)
        step8(session_id)
    print("\n=== Консультация завершена ===")


if __name__ == "__main__":
    main()

